In [1]:
import os, subprocess
import json
import uproot3
import awkward as ak
import numpy as np
from coffea import processor, util, hist
import pickle

from plotter import *

import warnings
warnings.filterwarnings('ignore')

In [2]:
lumis = {}
lumis['2016'] = 35.9
lumis['2017'] = 41.5
lumis['2018'] = 59.9

with open('xsec.json') as f:
  xs = json.load(f)

with open('pmap.json') as f:
  pmap = json.load(f)

systematics = [
    'PS_weightUp','PS_weightDown',
    'PDF_weightUp','PDF_weightDown',
    'scalevar_7ptUp','scalevar_7ptDown','scalevar_3ptUp','scalevar_3ptDown'
    ]

systematics_Wjets = [
    'W_d2kappa_EWDown','W_d2kappa_EWUp','W_d3kappa_EWDown','W_d3kappa_EWUp',
    'd1K_NLODown','d1K_NLOUp','d1kappa_EWDown','d1kappa_EWUp',
    'd2K_NLODown','d2K_NLOUp','d3K_NLODown','d3K_NLOUp',
    ]

systematics_Zjets = [
    'Z_d2kappa_EWDown','Z_d2kappa_EWUp','Z_d3kappa_EWDown','Z_d3kappa_EWUp',
    'd1K_NLODown','d1K_NLOUp','d1kappa_EWDown','d1kappa_EWUp',
    'd2K_NLODown','d2K_NLOUp','d3K_NLODown','d3K_NLOUp',
    ]

In [3]:
deta_cut = 3.5
mjj_cut = 1000

In [4]:
year = '2017'
nfiles = len(subprocess.getoutput("ls infiles-split/"+year+"*.json").split())
outsum = processor.dict_accumulator()

In [5]:
repickle=True

# Check if pickle exists, and don't re-create it if it does
picklename = 'pickles/'+str(year)+'_templates_th.pkl'
if os.path.isfile(picklename):
    repickle=False

In [ ]:
# Load all files - this takes a while
if repickle:
    nfiles = len(subprocess.getoutput("ls infiles-split/"+year+"*.json").split())
    for n in range(1,nfiles+1):

        with open('infiles-split/'+year+'_'+str(n)+'.json') as f:
            infiles = json.load(f)
    
        filename = '/myeosdir/vbf-theory/outfiles-ddb2/'+year+'_'+str(n)+'.coffea'
        if os.path.isfile(filename):
            out = util.load(filename)
            outsum.add(out)
        else:
            print('Missing file '+str(n),infiles.keys())
            #print("File " + filename + " is missing")
        
    scale_lumi = {k: xs[k] * 1000 *lumis[year] / w for k, w in outsum['sumw'].items()}
    outsum['templates-vbf'].scale(scale_lumi, 'dataset')
    templates = outsum['templates-vbf'].group('dataset', hist.Cat('process', 'Process'), pmap)

    outfile = open(picklename, 'wb')
    pickle.dump(templates, outfile, protocol=-1)
    outfile.close()

In [ ]:
# Read the histogram from the pickle file
templates = pickle.load(open(picklename,'rb'))
templates_vbf = templates.integrate('region','signal').integrate('deta',int_range=slice(deta_cut,7)).integrate('mjj',int_range=slice(mjj_cut,4000))

In [ ]:
templates_vbf.identifiers('systematic')

In [ ]:
os.system('rm '+year+'/1-signalregion-th.root')
fout = uproot3.create(year+'/1-signalregion-th.root')
for p in pmap.keys():  
    print(p)
    if "data" not in p:
        for s in systematics:
            h = templates_vbf.sum('pt1').integrate('systematic',s).integrate('ddb1',int_range=slice(0.89,1)).integrate('process',p)
            fout["pass_"+p+"_"+s] = hist.export1d(h)
            h = templates_vbf.sum('pt1').integrate('systematic',s).integrate('ddb1',int_range=slice(0,0.89)).integrate('process',p)
            fout["fail_"+p+"_"+s] = hist.export1d(h)
            
p = 'Wjets'
for s in systematics_Wjets:
    h = templates_vbf.sum('pt1').integrate('systematic',s).integrate('ddb1',int_range=slice(0.89,1)).integrate('process',p)
    fout["pass_"+p+"_"+s] = hist.export1d(h)
    h = templates_vbf.sum('pt1').integrate('systematic',s).integrate('ddb1',int_range=slice(0,0.89)).integrate('process',p)
    fout["fail_"+p+"_"+s] = hist.export1d(h)          
    
p = 'Zjets'
for s in systematics_Zjets:
    h = templates_vbf.sum('pt1').integrate('systematic',s).integrate('ddb1',int_range=slice(0.89,1)).integrate('process',p)
    fout["pass_"+p+"_"+s] = hist.export1d(h)
    h = templates_vbf.sum('pt1').integrate('systematic',s).integrate('ddb1',int_range=slice(0,0.89)).integrate('process',p)
    fout["fail_"+p+"_"+s] = hist.export1d(h)

fout.close()

In [ ]:
ptbins = [450, 550, 1200]
os.system('rm '+year+'/2pt-signalregion-th.root')
fout = uproot3.create(year+'/2pt-signalregion-th.root')

for i,b in enumerate(ptbins[:-1]):
    for p in pmap.keys(): 
        print(p)
        if "data" in p:
            s = "nominal"
            h = templates_vbf.integrate('systematic',s).integrate('pt1',int_range=slice(ptbins[i],ptbins[i+1])).integrate('ddb1',int_range=slice(0.89,1)).integrate('process',p)
            fout["pass_pt"+str(i+1)+"_"+p+"_"+s] = hist.export1d(h)
            h = templates_vbf.integrate('systematic',s).integrate('pt1',int_range=slice(ptbins[i],ptbins[i+1])).integrate('ddb1',int_range=slice(0,0.89)).integrate('process',p)
            fout["fail_pt"+str(i+1)+"_"+p+"_"+s] = hist.export1d(h)
        else:
            for s in systematics:
                h = templates_vbf.integrate('systematic',s).integrate('pt1',int_range=slice(ptbins[i],ptbins[i+1])).integrate('ddb1',int_range=slice(0.89,1)).integrate('process',p)
                fout["pass_pt"+str(i+1)+"_"+p+"_"+s] = hist.export1d(h)
                h = templates_vbf.integrate('systematic',s).integrate('pt1',int_range=slice(ptbins[i],ptbins[i+1])).integrate('ddb1',int_range=slice(0,0.89)).integrate('process',p)
                fout["fail_pt"+str(i+1)+"_"+p+"_"+s] = hist.export1d(h)
                
                
    p = 'Wjets'
    for s in systematics_Wjets:
        h = templates_vbf.integrate('systematic',s).integrate('pt1',int_range=slice(ptbins[i],ptbins[i+1])).integrate('ddb1',int_range=slice(0.89,1)).integrate('process',p)
        fout["pass_pt"+str(i+1)+"_"+p+"_"+s] = hist.export1d(h)
        h = templates_vbf.integrate('systematic',s).integrate('pt1',int_range=slice(ptbins[i],ptbins[i+1])).integrate('ddb1',int_range=slice(0,0.89)).integrate('process',p)
        fout["fail_pt"+str(i+1)+"_"+p+"_"+s] = hist.export1d(h)            
    
    p = 'Zjets'
    for s in systematics_Zjets:
        h = templates_vbf.integrate('systematic',s).integrate('pt1',int_range=slice(ptbins[i],ptbins[i+1])).integrate('ddb1',int_range=slice(0.89,1)).integrate('process',p)
        fout["pass_pt"+str(i+1)+"_"+p+"_"+s] = hist.export1d(h)
        h = templates_vbf.integrate('systematic',s).integrate('pt1',int_range=slice(ptbins[i],ptbins[i+1])).integrate('ddb1',int_range=slice(0,0.89)).integrate('process',p)
        fout["fail_pt"+str(i+1)+"_"+p+"_"+s] = hist.export1d(h) 
    

fout.close()

In [ ]:
mc = ['QCD','Wjets','Zjets','ttbar','singlet','VV','ggF','VBF','WH','ZH','ttH']

In [ ]:
h = templates.sum('pt1','ddb1').integrate('region','signal').integrate('deta',int_range=slice(3.5,7)).integrate('mjj',int_range=slice(1000,4000))
g = templates.sum('pt1').integrate('region','signal').integrate('deta',int_range=slice(3.5,7)).integrate('mjj',int_range=slice(1000,4000)).integrate('ddb1',int_range=slice(0.89,1))

In [ ]:
for p in mc:
    plot_syst(h.integrate('process',p),'PS_weight',year + ' ' + p + ' PS', year+'/syst_PS_weight/'+p)
    plot_syst(g.integrate('process',p),'PS_weight',year + ' ' + p + ' PS DDB pass', year+'/syst_PS_weight/'+p+"_pass_weight")

In [ ]:
for p in mc:
    plot_syst(h.integrate('process',p),'PDF_weight',year+' '+p + ' PDF',year+'/syst_PDF_weight/'+p)
    plot_syst(g.integrate('process',p),'PDF_weight',year + ' ' + p + ' PDF DDB pass', year+'/syst_PDF_weight/'+p+"_pass")

In [ ]:
for p in mc:
    plot_syst(h.integrate('process',p),'scalevar_7pt',year+' '+p + ' scale var 7pt',year+'/syst_scalevar_7pt/'+p)
    plot_syst(g.integrate('process',p),'scalevar_7pt',year + ' ' + p + ' scale var 7pt DDB pass', year+'/syst_scalevar_7pt/'+p+"_pass")

In [ ]:
for p in mc:
    plot_syst(h.integrate('process',p),'scalevar_3pt',year+' '+p + ' scale var 3pt',year+'/syst_scalevar_3pt/'+p)
    plot_syst(g.integrate('process',p),'scalevar_3pt',year + ' ' + p + ' scale var 3pt DDB pass', year+'/syst_scalevar_3pt/'+p+"_pass")

In [ ]:
for p in ['Wjets','Zjets']:
    plot_syst(h.integrate('process',p),'d1K_NLO',year+' '+p + ' d1K_NLO',year+'/syst_d1K_NLO/'+p)
    plot_syst(g.integrate('process',p),'d1K_NLO',year + ' ' + p + ' d1K_NLO DDB pass', year+'/syst_d1K_NLO/'+p+"_pass")

In [ ]:
for p in ['Wjets','Zjets']:
    plot_syst(h.integrate('process',p),'d1kappa_EW',year+' '+p + ' d1kappa_EW',year+'/syst_d1kappa_EW/'+p)
    plot_syst(g.integrate('process',p),'d1kappa_EW',year + ' ' + p + ' d1kappa_EW DDB pass', year+'/syst_d1kappa_EW/'+p+"_pass")

In [ ]:
for p in ['Wjets','Zjets']:
    plot_syst(h.integrate('process',p),'d2K_NLO',year+' '+p + ' d2K_NLO',year+'/syst_d2K_NLO/'+p)
    plot_syst(g.integrate('process',p),'d2K_NLO',year + ' ' + p + ' d2K_NLO DDB pass', year+'/syst_d2K_NLO/'+p+"_pass")

In [ ]:
for p in ['Wjets','Zjets']:
    plot_syst(h.integrate('process',p),'d3K_NLO',year+' '+p + ' d3K_NLO',year+'/syst_d3K_NLO/'+p)
    plot_syst(g.integrate('process',p),'d3K_NLO',year + ' ' + p + ' d3K_NLO DDB pass', year+'/syst_d3K_NLO/'+p+"_pass")

In [ ]:
for p in ['Wjets']:
    plot_syst(h.integrate('process',p),'W_d2kappa_EW',year+' '+p + ' W_d2kappa_EW',year+'/syst_W_d2kappa_EW/'+p)
    plot_syst(g.integrate('process',p),'W_d2kappa_EW',year + ' ' + p + ' W_d2kappa_EW DDB pass', year+'/syst_W_d2kappa_EW/'+p+"_pass")

In [ ]:
for p in ['Wjets']:
    plot_syst(h.integrate('process',p),'W_d3kappa_EW',year+' '+p + ' W_d3kappa_EW',year+'/syst_W_d3kappa_EW/'+p)
    plot_syst(g.integrate('process',p),'W_d3kappa_EW',year + ' ' + p + ' W_d3kappa_EW DDB pass', year+'/syst_W_d3kappa_EW/'+p+"_pass") 

In [ ]:
for p in ['Zjets']:
    plot_syst(h.integrate('process',p),'Z_d2kappa_EW',year+' '+p + ' Z_d2kappa_EW',year+'/syst_Z_d2kappa_EW/'+p)
    plot_syst(g.integrate('process',p),'Z_d2kappa_EW',year + ' ' + p + ' Z_d2kappa_EW DDB pass', year+'/syst_Z_d2kappa_EW/'+p+"_pass")

In [ ]:
for p in ['Zjets']:
    plot_syst(h.integrate('process',p),'Z_d3kappa_EW',year+' '+p + ' Z_d3kappa_EW',year+'/syst_Z_d3kappa_EW/'+p)
    plot_syst(g.integrate('process',p),'Z_d3kappa_EW',year + ' ' + p + ' Z_d3kappa_EW DDB pass', year+'/syst_Z_d3kappa_EW/'+p+"_pass") 